In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-02"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9647,2024-04-02,Taiwan P. League+,08:00,Fubon Braves,Hsinchu Jko Lioneers,1.43,2.68,192.5,1.85,1.85,0.0,0.00,0.00,jB07Fe3n,0.699301,0.373134,0.540541,0.540541,0.072435,195.544,72.851655,0.372559,0.6,1.341641,2.236068,318.24,2.078,0.621748,0.299205,-45.0,216.784,58.004235,0.267567,1.8,1.643168,0.912871,198.80,2.452,0.729226,0.297400,-19.0,102,71,3.12,2.80,151.608,254.434,0.430114,0.000000,NaN,-2.29,-0.458,-0.938865,0.644664,0.6,-0.044664,0.27,0.054,31.111111,0.385870,0.4,0.014130
9648,2024-04-02,Suécia Basketligan,14:04,Jamtland,Uppsala,1.10,6.48,163.5,1.87,1.87,0.0,0.00,0.00,pxOmJ80K,0.909091,0.154321,0.534759,0.534759,0.063412,158.334,58.474302,0.369310,2.4,1.341641,0.559017,97.79,1.872,0.719423,0.384307,37.0,180.432,42.587220,0.236029,1.2,1.643168,1.369306,247.10,2.268,0.856487,0.377640,-10.0,77,70,1.27,3.53,127.508,300.888,1.003756,0.000000,NaN,1.87,0.374,0.267380,0.722583,0.7,-0.022583,-3.49,-0.698,-7.851003,0.368129,0.3,-0.068129
9649,2024-04-02,Sérvia Primeira Liga,12:00,Joker,Zlatibor,1.54,2.37,172.5,1.81,1.89,0.0,0.00,0.00,OAR19KWc,0.649351,0.421941,0.552486,0.529101,0.071292,201.702,63.285024,0.313755,1.2,1.643168,1.369306,143.85,2.222,0.778698,0.350449,29.0,125.374,20.986154,0.167388,3.0,0.000000,0.000000,126.72,1.444,0.207437,0.143654,52.0,105,88,1.37,1.44,138.850,162.318,0.300204,0.030578,NaN,-0.76,-0.152,-3.552632,0.000000,0.0,0.000000,1.26,0.252,5.436508,0.600300,0.6,-0.000300
9650,2024-04-02,Sérvia Primeira Liga,11:30,OKK Beograd,Subotica,5.19,1.14,172.5,1.85,1.85,0.0,0.00,0.00,xKhqiwHT,0.192678,0.877193,0.540541,0.540541,0.069871,194.028,81.381865,0.419434,0.6,1.341641,2.236068,338.96,2.344,1.203611,0.513486,-40.0,119.738,8.006698,0.066868,3.0,0.000000,0.000000,128.82,1.258,0.105688,0.084013,83.0,76,113,4.46,1.14,146.994,107.046,0.904829,0.000000,NaN,-3.44,-0.688,-6.090116,0.639805,0.4,-0.239805,0.51,0.102,1.372549,0.000000,0.0,0.000000
9651,2024-04-02,Rússia Premier League Feminina,06:00,Enisey Krasnoyarsk F,MBA Moscow F,3.61,1.26,137.5,1.85,1.85,0.0,0.00,0.00,hro8kEK2,0.277008,0.793651,0.540541,0.540541,0.070659,289.518,219.464553,0.758034,0.6,1.341641,2.236068,223.82,4.448,3.470795,0.780305,-10.0,133.514,91.649460,0.686441,3.0,0.000000,0.000000,92.04,1.536,0.837753,0.545412,67.0,62,78,3.61,1.18,0.000,154.678,0.682423,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,1.35,0.270,0.962963,0.000000,0.0,0.000000
9652,2024-04-02,Lituânia Nkl,13:00,Jurbarkas,BC Olimpas Palanga,2.17,1.64,165.5,1.81,1.89,0.0,0.00,0.00,AuaVShmm,0.460829,0.609756,0.552486,0.529101,0.070586,184.800,70.278904,0.380297,2.4,1.341641,0.559017,308.00,1.898,0.686200,0.361538,21.0,110.830,11.747757,0.105998,1.8,1.643168,0.912871,125.96,1.186,0.101882,0.085904,41.0,100,94,3.08,1.34,164.358,125.676,0.196728,0.030578,NaN,-0.62,-0.124,-9.435484,0.530232,0.5,-0.030232,0.02,0.004,160.000000,0.718283,0.9,0.181717
9653,2024-04-02,Rússia Superliga,06:15,Dynamo Vladivostok,Barnaul,1.22,3.98,148.5,1.85,1.85,0.0,0.00,0.00,QuDPiyRs,0.819672,0.251256,0.540541,0.540541,0.070928,106.984,50.688475,0.473795,1.2,1.643168,1.369306,85.88,1.502,0.487770,0.324747,0.0,237.534,132.842667,0.559257,0.6,1.341641,2.236068,383.40,3.382,1.897438,0.561040,-22.0,76,71,1.13,5.40,97.346,235.634,0.750621,0.000000,NaN,-1.12,-0.224,-0.982143,0.000000,0.0,0.000000,-3.71,-0.7

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9654,10:00,República Tcheca Zbl Feminina,USK Prague F,Hradec Kralove F,1.61,Back Home
9683,20:30,Eua Nba,Miami Heat,New York Knicks,1.71,Back Home
9684,20:30,Eua Nba,Philadelphia 76ers,Oklahoma City Thunder,2.77,Back Home
9688,23:00,Eua Nba,Golden State Warriors,Dallas Mavericks,1.93,Back Home
